# Item based collaborative filtering python

In [1]:
#importing neccesary packages
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Step 1 - Data Preparation

In [2]:
#Reading movielens data from 
#https://grouplens.org/datasets/movielens/
my_df = pd.read_csv('C:\\Users\\User\\Downloads\\ml-100k\\u.data', sep='\t', names=['user_id','item_id','rating','timestamp'])

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\User\\Downloads\\ml-100k\\u.data'

In [ ]:
#Checking Data
my_df.head()

In [ ]:
#Importing movie title and joining with main data
movie_titles = pd.read_csv('C:\\Users\\User\\Downloads\\Movie_Titles.csv',encoding= 'unicode_escape')
my_df = pd.merge(my_df, movie_titles, on='item_id')
my_df.head()

# user_id - the ID of the user who rated the movie.
# item_id - the ID of the movie.
# rating - The rating the user gave the movie, between 1 and 5.
# timestamp - The time the movie was rated.
# title - The title of the movie.

# Step 2 - Data exploration

In [ ]:
#Statitical features of the movie
my_df.describe()

In [ ]:
#creating mean ratings data
ratings = pd.DataFrame(my_df.groupby('title')['rating'].mean())
ratings.head()

In [ ]:
#creating number of ratings data
ratings['number_of_ratings'] = my_df.groupby('title')['rating'].count()
ratings.head()

In [ ]:
#Plotting the jointplot
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.jointplot(x='rating', y='number_of_ratings', data=ratings)


# Creating User - Item interaction matrix

In [ ]:
#creating the user-item interaction matrix
movie_matrix_UII = my_df.pivot_table(index='user_id', columns='title', values='rating')
movie_matrix_UII.head()

In [ ]:
#Most rated movies
ratings.sort_values('number_of_ratings', ascending=False).head(10)

# Making recommendation - Example movie- FARGO

In [ ]:
#Fetching ratings for Fargo
Fargo_user_rating = movie_matrix_UII['Fargo (1996)']

In [ ]:
#Finding the correlation with different movies
similar_to_fargo=movie_matrix_UII.corrwith(Fargo_user_rating)

In [ ]:
#Observing the result
similar_to_fargo.head()

# creating a threshold for minimum number of ratings

In [ ]:
#creating dataframe to bring in #of ratings
corr_fargo = pd.DataFrame(similar_to_fargo, columns=['Correlation'])
corr_fargo.dropna(inplace=True)
corr_fargo.head()

In [ ]:
#Bringining in ratings
corr_fargo = corr_fargo.join(ratings['number_of_ratings'])

corr_fargo.head()

In [ ]:
corr_fargo[corr_fargo['number_of_ratings'] > 30].sort_values(by='Correlation', ascending=False).head(10)

# Next - User based recommendation - On Demand